Image Pre-Processing: Part 1 Crop the Image.
For the unprocessed images, just finish this step.

In [3]:
import numpy as np
import os
import cv2
from tqdm import tqdm
import imutils

def crop_img(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gray = cv2.GaussianBlur(gray, (3, 3), 0)
    thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=2)
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    c = max(cnts, key=cv2.contourArea)
    extLeft = tuple(c[c[:, :, 0].argmin()][0])
    extRight = tuple(c[c[:, :, 0].argmax()][0])
    extTop = tuple(c[c[:, :, 1].argmin()][0])
    extBot = tuple(c[c[:, :, 1].argmax()][0])
    ADD_PIXELS = 0
    new_img = img[extTop[1]-ADD_PIXELS:extBot[1]+ADD_PIXELS, extLeft[0]-ADD_PIXELS:extRight[0]+ADD_PIXELS].copy()
    return new_img

from google.colab import drive
drive.mount('/content/drive')

if __name__ == "__main__":
    base_dir = "/content/drive/My Drive"
    training = os.path.join(base_dir, "M209Data/Training")
    validation = os.path.join(base_dir, "M209Data/Validation")
    IMG_SIZE = 256

    print("Training directory contents:", os.listdir(training))
    print("Validation directory contents:", os.listdir(validation))

    for dir in os.listdir(training):
        save_path = os.path.join(base_dir, 'Cropped', dir)
        path = os.path.join(training, dir)
        image_dir = os.listdir(path)
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        for img in tqdm(image_dir):
            image_path = os.path.join(path, img)
            image = cv2.imread(image_path)
            new_img = crop_img(image)
            new_img = cv2.resize(new_img, (IMG_SIZE, IMG_SIZE))
            cv2.imwrite(os.path.join(save_path, img), new_img)

    for dir in os.listdir(validation):
        save_path = os.path.join(base_dir, 'Cropped', dir)
        path = os.path.join(validation, dir)
        image_dir = os.listdir(path)
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        for img in tqdm(image_dir):
            image_path = os.path.join(path, img)
            image = cv2.imread(image_path)
            new_img = crop_img(image)
            new_img = cv2.resize(new_img, (IMG_SIZE, IMG_SIZE))
            cv2.imwrite(os.path.join(save_path, img), new_img)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Training directory contents: ['pituitary', 'glioma', 'notumor', 'meningioma']
Validation directory contents: ['meningioma', 'notumor', 'pituitary', 'glioma']


100%|██████████| 300/300 [00:11<00:00, 25.98it/s]


Image Pre-Processing Part 2

In [5]:
import numpy as np
import cv2
import os

def preprocess_image(img_path, output_path):
    img = cv2.imread(img_path)
    if img is None:
        print("Failed to load image:", img_path)
        return

    #Salt and Pepper Noise Removal
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    median_filtered = cv2.medianBlur(gray_img, 5)

    #Edge Detection
    img_sobelx = cv2.Sobel(median_filtered, cv2.CV_8U, 1, 0, ksize=3)
    img_sobely = cv2.Sobel(median_filtered, cv2.CV_8U, 0, 1, ksize=3)
    img_sobel = img_sobelx + img_sobely + gray_img

    #Thresholding
    _, thresh = cv2.threshold(img_sobel, 50, 255, cv2.THRESH_BINARY)

    cv2.imwrite(output_path, img)

from google.colab import drive
drive.mount('/content/drive')

def process_directory(directory_path, output_directory):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    for root, dirs, files in os.walk(directory_path):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                file_path = os.path.join(root, file)
                output_path = os.path.join(output_directory, os.path.relpath(file_path, directory_path))
                output_file_directory = os.path.dirname(output_path)
                if not os.path.exists(output_file_directory):
                    os.makedirs(output_file_directory)
                preprocess_image(file_path, output_path)

base_dir = "/content/drive/My Drive/M209Data"
output_dir = "/content/drive/My Drive/Processed"

process_directory(base_dir, output_dir)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
